In [34]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bknbeQ86YP9H1I88aj26k1GJFEftfPgqj-0n7jcy-y7L',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-gb.cloud-object-storage.appdomain.cloud')

bucket = 'ibmhc-donotdelete-pr-cfvrz3cprlandb'
object_key = 'Placement_Data_Full_Class.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()
import numpy as np
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [35]:
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay,classification_report
warnings.filterwarnings('ignore')

In [36]:
""""df=pd.read_csv("/content/Placement_Data_Full_Class.csv")
df"""

'"df=pd.read_csv("/content/Placement_Data_Full_Class.csv")\ndf'

In [37]:
df['status'].value_counts()

Placed        148
Not Placed     67
Name: status, dtype: int64

**Data Preprocessing**

In [38]:
df.shape

(215, 15)

In [39]:
df.dtypes

sl_no               int64
gender             object
ssc_p             float64
ssc_b              object
hsc_p             float64
hsc_b              object
hsc_s              object
degree_p          float64
degree_t           object
workex             object
etest_p           float64
specialisation     object
mba_p             float64
status             object
salary            float64
dtype: object

In [40]:
df.isna().sum()

sl_no              0
gender             0
ssc_p              0
ssc_b              0
hsc_p              0
hsc_b              0
hsc_s              0
degree_p           0
degree_t           0
workex             0
etest_p            0
specialisation     0
mba_p              0
status             0
salary            67
dtype: int64

In [41]:
df1=df.copy()
df1 = df1.drop(['sl_no','ssc_b','hsc_b'],axis=1)
df1

,gender,ssc_p,hsc_p,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,M,67.00,91.00,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,M,79.33,78.33,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,M,65.00,68.00,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,M,56.00,52.00,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,80.60,82.00,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,M,58.00,60.00,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,M,67.00,67.00,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,F,74.00,66.00,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [42]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lst=['gender','hsc_s','degree_t','workex','specialisation','status']
for i in lst:
  df1[i]=le.fit_transform(df1[i])

In [43]:
df1

,gender,ssc_p,hsc_p,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,67.00,91.00,1,58.00,2,0,55.0,1,58.80,1,270000.0
1,1,79.33,78.33,2,77.48,2,1,86.5,0,66.28,1,200000.0
2,1,65.00,68.00,0,64.00,0,0,75.0,0,57.80,1,250000.0
3,1,56.00,52.00,2,52.00,2,0,66.0,1,59.43,0,NaN
4,1,85.80,73.60,1,73.30,0,0,96.8,0,55.50,1,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,82.00,1,77.60,0,0,91.0,0,74.49,1,400000.0
211,1,58.00,60.00,2,72.00,2,0,74.0,0,53.62,1,275000.0
212,1,67.00,67.00,1,73.00,0,1,59.0,0,69.72,1,295000.0
213,0,74.00,66.00,1,58.00,0,0,70.0,1,60.23,1,204000.0


In [44]:
df1.dtypes

gender              int64
ssc_p             float64
hsc_p             float64
hsc_s               int64
degree_p          float64
degree_t            int64
workex              int64
etest_p           float64
specialisation      int64
mba_p             float64
status              int64
salary            float64
dtype: object

In [45]:
#Filling the missing values in the column salary
df1['salary']=df1['salary'].fillna(df1['salary'].mode()[0])

In [46]:
X=df1.iloc[:,:10]
y=df1.iloc[:,-2]

In [47]:
#scaling the dataset using Min-Max scaler
from sklearn.preprocessing import MinMaxScaler
ms=MinMaxScaler()
X_sc=ms.fit_transform(X)
X_sc

array([[1.        , 0.53823954, 0.88962109, ..., 0.10416667, 1.        ,
        0.28448276],
       [1.        , 0.79241394, 0.68088962, ..., 0.76041667, 0.        ,
        0.56484258],
       [1.        , 0.49701093, 0.5107084 , ..., 0.52083333, 0.        ,
        0.2470015 ],
       ...,
       [1.        , 0.53823954, 0.49423394, ..., 0.1875    , 0.        ,
        0.69377811],
       [0.        , 0.68253968, 0.47775947, ..., 0.41666667, 1.        ,
        0.33808096],
       [1.        , 0.43516801, 0.34596376, ..., 0.8125    , 1.        ,
        0.33770615]])

In [48]:
#splitting the dataset to train data and test data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_sc,y,test_size=0.2,random_state=10)

**K Neighbors classifier**

In [49]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.44      0.53        18
           1       0.68      0.84      0.75        25

    accuracy                           0.67        43
   macro avg       0.67      0.64      0.64        43
weighted avg       0.67      0.67      0.66        43



**Support Vector Classifier**

In [50]:
svc=SVC()
svc.fit(X_train,y_train)
y_pred1=svc.predict(X_test)
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.82      0.50      0.62        18
           1       0.72      0.92      0.81        25

    accuracy                           0.74        43
   macro avg       0.77      0.71      0.71        43
weighted avg       0.76      0.74      0.73        43



**Naive Bayes**

In [51]:
nb=GaussianNB()
nb.fit(X_train,y_train)
y_pred2=nb.predict(X_test)
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.69      0.61      0.65        18
           1       0.74      0.80      0.77        25

    accuracy                           0.72        43
   macro avg       0.71      0.71      0.71        43
weighted avg       0.72      0.72      0.72        43



**Decision Tree**

In [52]:
dt=DecisionTreeClassifier(criterion='gini',random_state=1)
dt.fit(X_train,y_train)
y_pred3=dt.predict(X_test)
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

           0       0.55      0.33      0.41        18
           1       0.62      0.80      0.70        25

    accuracy                           0.60        43
   macro avg       0.59      0.57      0.56        43
weighted avg       0.59      0.60      0.58        43



**Random Forest**

In [53]:
rf=RandomForestClassifier(n_estimators=100,random_state=1)
rf.fit(X_train,y_train)
y_pred4=rf.predict(X_test)
print(classification_report(y_test,y_pred4))

              precision    recall  f1-score   support

           0       0.92      0.61      0.73        18
           1       0.77      0.96      0.86        25

    accuracy                           0.81        43
   macro avg       0.85      0.79      0.80        43
weighted avg       0.83      0.81      0.81        43



**AdaBoost**

In [54]:
ad=AdaBoostClassifier()
ad.fit(X_train,y_train)
y_pred5=ad.predict(X_test)
print(classification_report(y_test,y_pred5))

              precision    recall  f1-score   support

           0       0.88      0.78      0.82        18
           1       0.85      0.92      0.88        25

    accuracy                           0.86        43
   macro avg       0.86      0.85      0.85        43
weighted avg       0.86      0.86      0.86        43



**Gradient Boosting**

In [55]:
gb=GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred6=gb.predict(X_test)
print(classification_report(y_test,y_pred6))

              precision    recall  f1-score   support

           0       0.90      0.50      0.64        18
           1       0.73      0.96      0.83        25

    accuracy                           0.77        43
   macro avg       0.81      0.73      0.74        43
weighted avg       0.80      0.77      0.75        43



**XG boosting**

In [56]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred7=xgb.predict(X_test)
print(classification_report(y_test,y_pred7))

              precision    recall  f1-score   support

           0       0.92      0.67      0.77        18
           1       0.80      0.96      0.87        25

    accuracy                           0.84        43
   macro avg       0.86      0.81      0.82        43
weighted avg       0.85      0.84      0.83        43



Here we can see the accuracy score of AdaBoost has an accuracy of 93%

In [57]:
y.value_counts()

1    148
0     67
Name: status, dtype: int64

Here we can see that the target variable is highly imbalanced. Hence applying oversampling method which is Random **OverSampler**

In [58]:
pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 20.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [59]:
from imblearn.over_sampling import RandomOverSampler
sm = RandomOverSampler()
X_train_sm,y_train_sm=sm.fit_resample(X_train,y_train)

In [60]:
y_train_sm.value_counts()

1    123
0    123
Name: status, dtype: int64

**K Neighbors classifier**

In [61]:
Knn_sm=KNeighborsClassifier()
Knn_sm.fit(X_train_sm,y_train_sm)
y_pred_sm=Knn_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm))

              precision    recall  f1-score   support

           0       0.53      0.56      0.54        18
           1       0.67      0.64      0.65        25

    accuracy                           0.60        43
   macro avg       0.60      0.60      0.60        43
weighted avg       0.61      0.60      0.61        43



**Support Vector Machine**

In [62]:
sv_sm=SVC()
sv_sm.fit(X_train_sm,y_train_sm)
y_pred_sm1=sv_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm1))

              precision    recall  f1-score   support

           0       0.67      0.56      0.61        18
           1       0.71      0.80      0.75        25

    accuracy                           0.70        43
   macro avg       0.69      0.68      0.68        43
weighted avg       0.69      0.70      0.69        43



**Naive Bayes**

In [63]:
nb_sm=GaussianNB()
nb_sm.fit(X_train_sm,y_train_sm)
y_pred_sm2=nb_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm2))

              precision    recall  f1-score   support

           0       0.60      0.67      0.63        18
           1       0.74      0.68      0.71        25

    accuracy                           0.67        43
   macro avg       0.67      0.67      0.67        43
weighted avg       0.68      0.67      0.68        43



**Decision Tree Classifier**

In [64]:
dt_sm=DecisionTreeClassifier(criterion='entropy')
dt_sm.fit(X_train_sm,y_train_sm)
y_pred_sm3=dt_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm3))

              precision    recall  f1-score   support

           0       0.83      0.56      0.67        18
           1       0.74      0.92      0.82        25

    accuracy                           0.77        43
   macro avg       0.79      0.74      0.74        43
weighted avg       0.78      0.77      0.76        43



**Random Forest Classifier**

In [65]:
rf_sm=RandomForestClassifier(n_estimators=100,random_state=1)
rf_sm.fit(X_train_sm,y_train_sm)
y_pred_sm4=rf_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm4))

              precision    recall  f1-score   support

           0       0.93      0.72      0.81        18
           1       0.83      0.96      0.89        25

    accuracy                           0.86        43
   macro avg       0.88      0.84      0.85        43
weighted avg       0.87      0.86      0.86        43



**Adaboost**

In [66]:
ad_sm=AdaBoostClassifier()
ad_sm.fit(X_train_sm,y_train_sm)
y_pred_sm5=ad_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm5))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80        18
           1       0.85      0.88      0.86        25

    accuracy                           0.84        43
   macro avg       0.83      0.83      0.83        43
weighted avg       0.84      0.84      0.84        43



**Gradient Boosting**

In [67]:
gb_sm=GradientBoostingClassifier()
gb_sm.fit(X_train_sm,y_train_sm)
y_pred_sm6=gb_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm6))

              precision    recall  f1-score   support

           0       0.79      0.61      0.69        18
           1       0.76      0.88      0.81        25

    accuracy                           0.77        43
   macro avg       0.77      0.75      0.75        43
weighted avg       0.77      0.77      0.76        43



**XG boost**

In [68]:
from sklearn.metrics import confusion_matrix

In [69]:
confusion_matrix(y_test, y_pred_sm)

array([[10,  8],
       [ 9, 16]])

In [70]:
confusion_matrix(y_test, y_pred_sm1)

array([[10,  8],
       [ 5, 20]])

In [71]:
confusion_matrix(y_test, y_pred_sm2)

array([[12,  6],
       [ 8, 17]])

In [72]:
confusion_matrix(y_test, y_pred_sm3)

array([[10,  8],
       [ 2, 23]])

In [73]:
confusion_matrix(y_test, y_pred_sm4)

array([[13,  5],
       [ 1, 24]])

In [74]:
confusion_matrix(y_test, y_pred_sm5)

array([[14,  4],
       [ 3, 22]])

In [75]:
confusion_matrix(y_test, y_pred_sm6)

array([[11,  7],
       [ 3, 22]])

In [76]:
xgb_sm=XGBClassifier()
xgb_sm.fit(X_train_sm,y_train_sm)
y_pred_sm7=xgb_sm.predict(X_test)
print(classification_report(y_test,y_pred_sm7))

              precision    recall  f1-score   support

           0       0.73      0.61      0.67        18
           1       0.75      0.84      0.79        25

    accuracy                           0.74        43
   macro avg       0.74      0.73      0.73        43
weighted avg       0.74      0.74      0.74        43



In [77]:
confusion_matrix(y_test, y_pred_sm7)

array([[11,  7],
       [ 4, 21]])

In [78]:
import pickle

In [79]:
pickle.dump(gb_sm,open('classification_gb_sm.pkl','wb'))

In [80]:
!pip install -U ibm-watson-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.9 MB/s eta 0:00:0000:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.311
    Uninstalling ibm-watson-machine-learning-1.0.311:
      Successfully uninstalled ibm-watson-machine-learning-1.0.311


In [81]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [82]:
wml_credentials = {
    "apikey": "u3NGWPGvt98WfwbStWuqtaTdI-4GdYieODsBdz9vo3Yr",
    "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [83]:
wml_client = APIClient(wml_credentials)

In [87]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------------  ------------------------
ID                                    NAME           CREATED
be95cabb-cdba-4209-a59c-5a2b9004691f  IBM_HC_DEPLOY  2023-08-05T07:24:33.150Z
------------------------------------  -------------  ------------------------


,ID,NAME,CREATED
0,be95cabb-cdba-4209-a59c-5a2b9004691f,IBM_HC_DEPLOY,2023-08-05T07:24:33.150Z


In [88]:
SPACE_ID="be95cabb-cdba-4209-a59c-5a2b9004691f"

In [89]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [104]:
MODEL_NAME = "DemoMODEL"
DEPLOYMENT_NAME = "demoModelDep"
BEST_MODEL = gb_sm

In [105]:
#set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.2-py3.10')

In [106]:
#Setup Model Meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.1',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

In [111]:
#Save model
model_details = wml_client.repository.store_model(
    model= BEST_MODEL,
    meta_props = model_props,
    training_data=X_train,
    training_target=y_train)

In [112]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'status',
  'schemas': {'input': [{'fields': [{'name': 'f0', 'type': 'float'},
      {'name': 'f1', 'type': 'float'},
      {'name': 'f2', 'type': 'float'},
      {'name': 'f3', 'type': 'float'},
      {'name': 'f4', 'type': 'float'},
      {'name': 'f5', 'type': 'float'},
      {'name': 'f6', 'type': 'float'},
      {'name': 'f7', 'type': 'float'},
      {'name': 'f8', 'type': 'float'},
      {'name': 'f9', 'type': 'float'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': 'b56101f1-309d-549b-a849-eaa63f77b2fb',
   'name': 'runtime-22.2-py3.10'},
  'training_data_references': [{'id': '1',
    'location': {},
    'schema': {'fields': [{'name': 'f0', 'type': 'float'},
      {'name': 'f1', 'type': 'float'},
      {'name': 'f2', 'type': 'float'},
      {'name': 'f3', 'type': 'float'},
      {'name': 'f4', 'type': 'float'},
      {'name': 'f5', 'type': 'float'},
      {'name': 'f6', 'ty

In [113]:
model_uid = wml_client.repository.get_model_id(model_details)
model_uid

'0520677f-d7c3-4d55-9d82-6884e02c7e87'

In [116]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [117]:
#DEPLOY
Deployment1 = wml_client.deployments.create(
    artifact_uid = model_uid,
    meta_props = deployment_props)



#######################################################################################

Synchronous deployment creation for uid: '0520677f-d7c3-4d55-9d82-6884e02c7e87' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1bf8f596-e9a4-4519-823a-2ca2c2ba50d3'
------------------------------------------------------------------------------------------------


